In [119]:
def read_file(path):
    res = open(path, "r").readlines()
    res = [x.strip() for x in res]
    map = []
    instructions = []
    is_map = True
    for l in res:
        if l == "":
            is_map = False

        if is_map:
            map.append(l)
        else:
            instructions.append(l)

    map = [list(x) for x in map]
    instructions = [list(x) for x in instructions]
    instructions = [x for xs in instructions for x in xs]
    return map, instructions


def read_file2(path):
    res = open(path, "r").readlines()
    res = [
        x.strip()
        .replace("#", "##")
        .replace("O", "[]")
        .replace(".", "..")
        .replace("@", "@.")
        for x in res
    ]
    map = []
    instructions = []
    is_map = True
    for l in res:
        if l == "":
            is_map = False

        if is_map:
            map.append(l)
        else:
            instructions.append(l)

    map = [list(x) for x in map]
    instructions = [list(x) for x in instructions]
    instructions = [x for xs in instructions for x in xs]
    return map, instructions


input = read_file("input")

In [120]:
DIRECTIONS = {"<": (0, -1), "^": (-1, 0), "v": (1, 0), ">": (0, 1)}

In [121]:
input

([['#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#'],
  ['#',
   '#',
   '.',
   '.',
   '.',
   'O',
   '.',
   '.',
   '.',
   '#',
   'O',
   'O',
   '.',
   '.',
   '.',
   'O',
   '.',
   '#',
   '.',
   '.',
   '.',
   '.',
   '#',
   '.',
   '.',
   '.',
   '.',
   '#',
   'O',
   '#',
   'O',
   '.',
   '.',
   'O',
   '.',
   '.',
   'O',
   '.',
   '.',
   '.',
   '.',
   '.',
   '.',
   'O',
   '.',
   '.',
   'O',
   '.',
   '.',
   '#'],
  ['#',
   'O',
   'O',
   'O',
   'O',
   '.',
   '#',
   '.',
   'O',
   '.',
   'O',
   '.',
   '.',
   '.',
   '.',
   '.',
   'O',
   'O',
   '#',
   '.',
   '.',
   '.',
   'O',
   '.',
   'O'

In [242]:
from copy import deepcopy


def is_move_possible(grid, x, y, direction):
    dx, dy = DIRECTIONS[direction]
    x, y = x + dx, y + dy
    grid_val = grid[x][y]
    if grid_val == ".":
        return True
    if grid_val == "#":
        return False
    else:
        return is_move_possible(grid, x, y, direction)


def move(grid, x, y, direction):
    if not is_move_possible(grid, x, y, direction):
        return grid

    moving = grid[x][y]
    dx, dy = DIRECTIONS[direction]
    nx, ny = x + dx, y + dy
    if grid[nx][ny] == ".":
        grid[x][y] = "."
        grid[nx][ny] = moving
    else:
        move(grid, nx, ny, direction)
        move(grid, x, y, direction)
    return grid


def display_grid(grid):
    print("\n\n")
    print("    " + "".join([f"{str(x):<2}" for x in range(len(grid[0]))]))
    for i, al in enumerate(grid):
        l = str(i) + "   " + " ".join(al)
        print(l)


def score_grid(grid):
    res = []
    for x, l in enumerate(grid):
        for y, c in enumerate(l):
            if c in ["O", "["]:
                res.append(x * 100 + y)
    return res


def find_robot(grid):
    for x, l in enumerate(grid):
        for y, c in enumerate(l):
            if c == "@":
                return x, y


def simulate(grid, instructions):
    grid = deepcopy(grid)
    for i, ins in enumerate(instructions):
        # print(f"Running instruction {i} - {ins}")
        x, y = find_robot(grid)
        grid = move(grid, x, y, ins)
    return grid

In [123]:
grid = simulate(input[0], input[1])
# display_grid(grid)

In [124]:
sum(score_grid(grid))

1406628

In [240]:
def is_move_possible2(grid, objects_to_move: list, direction: str):
    dx, dy = DIRECTIONS[direction]
    can_move = True
    for x, y in objects_to_move:
        nx, ny = x + dx, y + dy
        next_spot = grid[nx][ny]

        if next_spot == "#":
            return False
        elif next_spot == ".":
            continue
        elif direction in ["<", ">"]:
            can_move = can_move and is_move_possible2(grid, [(nx, ny)], direction)
        elif next_spot == "[":
            can_move = can_move and is_move_possible2(grid, [(nx, ny)], direction)
            can_move = can_move and is_move_possible2(grid, [(nx, ny + 1)], direction)
        elif next_spot == "]":
            can_move = can_move and is_move_possible2(grid, [(nx, ny)], direction)
            can_move = can_move and is_move_possible2(grid, [(nx, ny - 1)], direction)
        else:
            raise Exception(
                f"This should not be possible. Objects_to_move {objects_to_move} in direction {direction}"
            )

    return can_move


def move2(grid, objects_to_move: list, direction: str):
    if not is_move_possible2(grid, objects_to_move, direction):
        return grid
    grid = deepcopy(grid)

    dx, dy = DIRECTIONS[direction]
    for x, y in objects_to_move:
        nx, ny = x + dx, y + dy
        next_space = grid[nx][ny]
        if next_space == ".":
            # print(f"Moving into . from {x, y} in direction {direction} to {nx, ny}")
            grid[nx][ny] = grid[x][y]
            grid[x][y] = "."
        elif direction in ["<", ">"]:
            grid = move2(grid, [(nx, ny)], direction)
            grid = move2(grid, [(x, y)], direction)
        elif next_space == "[":
            grid = move2(grid, [(nx, ny), (nx, ny + 1)], direction)
            grid = move2(grid, [(x, y)], direction)
        elif next_space == "]":
            grid = move2(grid, [(nx, ny), (nx, ny - 1)], direction)
            grid = move2(grid, [(x, y)], direction)
        else:
            raise Exception(
                f"This should not happen. Objects to move {objects_to_move} in direction {direction}."
            )
    return grid


def simulate2(grid, instructions):
    grid = deepcopy(grid)
    for i, ins in enumerate(instructions):
        # print(f"Running instruction {i} - {ins}")
        x, y = find_robot(grid)
        # print(f"About to move {x,y} in direction {ins}")
        # display_grid(grid)
        grid = move2(grid, [(x, y)], ins)
        # print("After move")
        # display_grid(grid)
    return grid

In [206]:
import sys

sys.setrecursionlimit(9999)
sys.getrecursionlimit()

9999

In [245]:
input2 = list(read_file2("input"))
grid2 = input2[0]
instructions = input2[1]
display_grid(grid2)
grid2 = simulate2(input2[0], input2[1])
display_grid(grid2)
sum(score_grid(grid2))




    0 1 2 3 4 5 6 7 8 9 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899
0   # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
1   # # # # . . . . . . [ ] . . . . . . # # [ ] [ ] . . . . . . [ ] . . # # . . . . . . . . # # . . . . . . . . # # [ ] # # [ ] . . . . [ ] . . . . [ ] . . . . . . . . . . . . [ ] . . . . [ ] . . . . # #
2   # # [ ] [ ] [ ] [ ] . . # # . . [ ] . . [ ] . . . . . . . . . . [ ] [ ] # # . . . . . . [ ] . . [ ] [ ] . . [ ] . . . . . . . . [ ] . . # # . . . . [ ] [ ] . . . . . . . . [ ] # # [ ] [ ] [ ] . . # #
3   # # . . . . . . # # . . # # [ ] . . . . [ ] . . . . . . [ ] [ ] . . . . . . . . . . . . [ ] . . . . . . . . . . . . . . # # . . . . . . . . [ ] . . . . [ ] # # . . [ ] . . . . 

1432781